### Monitor models

_Application Insights_ is an application performance management service in Microsoft Azure that enables the capture, storage, and analysis of telemetry data from applications.

When you've deployed a model into production as a service, you'll want to monitor it to track usage and explore the requests it processes. You can use Azure Application Insights to monitor activity for a model service endpoint.

Application Insights automatically captures any information written to the standard output and error logs, and provides a query capability to view data in these logs.



In [8]:
from azureml.core import Workspace

ws = Workspace.from_config()

First of all, create and register a model.

In [11]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from azureml.core import Dataset

# Upload data files to the default datastore
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['Script/data/diabetes.csv'],
                       target_path='diabetes-data/',
                       overwrite=True,
                       show_progress=True)

#Create a tabular dataset from the path on the datastore
print('Creating dataset...')
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))
# Register the tabular dataset
print('Registering dataset...')
try:
    data_set = data_set.register(workspace=ws,
                               name='diabetes dataset',
                               description='diabetes data',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

Uploading an estimated of 1 files
Uploading Script/data/diabetes.csv
Uploaded Script/data/diabetes.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Creating dataset...
Registering dataset...


In [12]:
# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')

run = experiment.start_logging()

print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = data_set.to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
print("Split Data...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
print('Save data...')
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
print('Registering model...')
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# Get the registered model
model = ws.models['diabetes_model']

print('Model trained and registered.')

Starting experiment: mslearn-train-diabetes
Loading Data...
Split Data...
Training a decision tree model
Accuracy: 0.8893333333333333
AUC: 0.8741628936763023
Save data...


C:\Users\ravazzil\Anaconda3\envs\azure_env\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\ravazzil\Anaconda3\envs\azure_env\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Registering model...
Model trained and registered.


Deploy the registered model as a service.

In [31]:
import os

deployment_folder = './Script/diabetes_service_app'
os.makedirs(deployment_folder, exist_ok=True)

script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder, script_file)

In [32]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [33]:
service_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
service_env.inferencing_stack_version="latest"

In [26]:
deployment_folder

'./Script/diabetes_service_appinsight'

In [34]:
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=aci_service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [35]:
# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service-app-insights"
aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

Deploying model...


C:\Users\ravazzil\Anaconda3\envs\azure_env\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  after removing the cwd from sys.path.


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-01-18 17:21:14+01:00 Creating Container Registry if not exists.
2023-01-18 17:21:14+01:00 Registering the environment.
2023-01-18 17:21:16+01:00 Building image..
2023-01-18 17:31:31+01:00 Generating deployment configuration..
2023-01-18 17:31:32+01:00 Submitting deployment to compute..
2023-01-18 17:31:39+01:00 Checking the status of deployment diabetes-service-app-insights..
2023-01-18 17:33:32+01:00 Checking the status of inference endpoint diabetes-service-app-insights.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [36]:
# Enable app insight.
aci_service.update(enable_app_insights = True)

In [38]:
import requests
import json

# Create new data for inferencing
x_new = [[1,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

# Get the predictions
endpoint = aci_service.scoring_uri
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

200
Patient [1, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


To view logs, you have to follow these steps:

- In the Azure portal, open your Machine Learning workspace.
- On the Overview page, click the link for the associated Application Insights resource.
- On the Application Insights blade, click Logs.
- Query your logs.

In [ ]:
try:
    aci_service.delete()
    print('Service deleted.')
except Exception as ex:
    print(ex.message)